In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from collections import defaultdict

from custom_dataset import IND2CLASS
from SAM2UNet import SAM2UNet

# --- [RLE 인코딩 함수] ---
def encode_mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# --- [데이터셋 클래스] ---
class XRayInferenceDataset(Dataset):
    def __init__(self, image_root, transforms=None):
        self.image_root = image_root
        
        # 1. 파일 리스트 추출
        pngs = {
            os.path.relpath(os.path.join(root, fname), start=image_root)
            for root, _dirs, files in os.walk(image_root)
            for fname in files
            if os.path.splitext(fname)[1].lower() == ".png"
        }
        self.filenames = np.array(sorted(list(pngs)))
        self.transforms = transforms
        
        # 2. Hand Side (Left/Right) 판별 로직
        # 같은 폴더 내 파일명 정렬 시: 첫 번째=Right, 두 번째=Left 가정
        self.hand_side_map = {}
        files_by_folder = defaultdict(list)
        for fname in self.filenames:
            folder = os.path.dirname(fname)
            files_by_folder[folder].append(fname)
            
        for folder, files in files_by_folder.items():
            files.sort()
            if len(files) > 0: self.hand_side_map[files[0]] = 'Right'
            if len(files) > 1: self.hand_side_map[files[1]] = 'Left'
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        image_name = self.filenames[item]
        image_path = os.path.join(self.image_root, image_name)
        
        # 1. 이미지 로드 (Gray)
        image_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # 2. [전처리] Right -> Left로 좌우 반전 (모델은 Left만 학습했으므로)
        hand_side = self.hand_side_map.get(image_name, 'Unknown')
        if hand_side == 'Right':
            image_gray = cv2.flip(image_gray, 1)
        
        # 3. 3채널 변환 및 정규화
        image = np.stack([image_gray]*3, axis=-1)
        image = image / 255.0
        image = image.astype(np.float32)
        
        if self.transforms:
            image = self.transforms(image=image)["image"]

        # 4. To Tensor (H, W, C) -> (C, H, W)
        if isinstance(image, np.ndarray):
            image = image.transpose(2, 0, 1)
            image = torch.from_numpy(image).float()
            
        return image, image_name

In [ ]:
IMAGE_ROOT = "../data/test/DCM"
CHECKPOINT_PATH = "../sam2_unet_result_checkpoints/experiment20.pth"
HIERA_PATH = "../checkpoints/sam2_hiera_large.pt"
SAVE_PATH = "result_full_size.csv"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1

In [ ]:
def test_full_size(model, data_loader, thr=0.5):
    model.eval()
    rles = []
    filename_and_class = []
    dataset = data_loader.dataset
    
    print("🚀 [Full Size] Inference Start...")
    
    with torch.no_grad():
        for images, image_names in tqdm(data_loader):
            images = images.to(DEVICE)
            image_name = image_names[0]
            
            # 1. 추론
            preds = model(images)
            if isinstance(preds, tuple): preds = preds[0]
            
            # 2. 크기 보정 (혹시 2048이 아닐 경우)
            if preds.shape[-1] != 2048:
                preds = F.interpolate(preds, size=(2048, 2048), mode="bilinear", align_corners=False)
            
            # 3. [Re-flip] Right 손인 경우, 다시 원래대로 반전
            hand_side = dataset.hand_side_map.get(image_name, 'Unknown')
            if hand_side == 'Right':
                preds = torch.flip(preds, dims=[-1])
            
            # 4. Thresholding
            preds = torch.sigmoid(preds)
            preds = (preds > thr).detach().cpu().numpy()
            
            # 5. RLE Encoding
            for output, img_name in zip(preds, image_names):
                for c, segm in enumerate(output):
                    rle = encode_mask_to_rle(segm)
                    rles.append(rle)
                    filename_and_class.append(f"{IND2CLASS[c]}_{img_name}")
                    
    return rles, filename_and_class

In [ ]:
WINDOW_SIZE = 1024
STRIDE = 512

def sliding_window_inference(model, image, window_size, stride, num_classes=29):
    model.eval()
    _, _, H, W = image.shape
    prob_map = torch.zeros((1, num_classes, H, W), device=DEVICE)
    count_map = torch.zeros((1, num_classes, H, W), device=DEVICE)
    
    # 스텝 설정 (마지막 자투리 영역 포함)
    h_steps = list(range(0, H - window_size + 1, stride))
    w_steps = list(range(0, W - window_size + 1, stride))
    if (H - window_size) % stride != 0: h_steps.append(H - window_size)
    if (W - window_size) % stride != 0: w_steps.append(W - window_size)

    with torch.no_grad():
        for h in h_steps:
            for w in w_steps:
                img_patch = image[:, :, h:h+window_size, w:w+window_size]
                preds = model(img_patch)
                if isinstance(preds, tuple): preds = preds[0]
                
                prob_patch = torch.sigmoid(preds)
                prob_map[:, :, h:h+window_size, w:w+window_size] += prob_patch
                count_map[:, :, h:h+window_size, w:w+window_size] += 1.0
                
    return (prob_map / count_map).squeeze(0) # (C, H, W)

def test_sliding(model, data_loader, thr=0.5):
    rles = []
    filename_and_class = []
    dataset = data_loader.dataset
    
    print("🚀 [Sliding Window] Inference Start...")
    
    for images, image_names in tqdm(data_loader):
        images = images.to(DEVICE)
        image_name = image_names[0]
        
        # 1. Sliding Window 추론
        probs = sliding_window_inference(model, images, WINDOW_SIZE, STRIDE)
        
        # 2. [Re-flip] Right 손인 경우 복구
        hand_side = dataset.hand_side_map.get(image_name, 'Unknown')
        if hand_side == 'Right':
            probs = torch.flip(probs, dims=[-1])
            
        # 3. Thresholding & RLE
        preds = (probs > thr).detach().cpu().numpy()
        
        for c, segm in enumerate(preds):
            rle = encode_mask_to_rle(segm)
            rles.append(rle)
            filename_and_class.append(f"{IND2CLASS[c]}_{image_name}")
            
    return rles, filename_and_class

In [ ]:
MODEL_CONFIGS = [
    {
        "name": "sam_sliding",
        "type": "sam2unet",
        "path": "../sam2_unet_result_checkpoints/experiment16.pth",
        "method": "sliding",
        "weight": 0.5 
    },
    {
        "name": "sam_full",
        "type": "sam2unet",
        "path": "../sam2_unet_result_checkpoints/experiment20.pth",
        "method": "full",
        "weight": 0.5
    },
]

# --- [추론 헬퍼 함수] ---
def sliding_window_inference(model, image, window_size=1024, stride=512, num_classes=29):
    # (위의 슬라이딩 윈도우 함수와 동일)
    model.eval()
    _, _, H, W = image.shape
    prob_map = torch.zeros((1, num_classes, H, W), device=DEVICE)
    count_map = torch.zeros((1, num_classes, H, W), device=DEVICE)
    
    h_steps = list(range(0, H - window_size + 1, stride))
    w_steps = list(range(0, W - window_size + 1, stride))
    if (H - window_size) % stride != 0: h_steps.append(H - window_size)
    if (W - window_size) % stride != 0: w_steps.append(W - window_size)

    with torch.no_grad():
        for h in h_steps:
            for w in w_steps:
                img_patch = image[:, :, h:h+window_size, w:w+window_size]
                preds = model(img_patch)
                if isinstance(preds, tuple): preds = preds[0]
                prob_map[:, :, h:h+window_size, w:w+window_size] += torch.sigmoid(preds)
                count_map[:, :, h:h+window_size, w:w+window_size] += 1.0
                
    return (prob_map / count_map).squeeze(0)

def ensemble_test(models_list, data_loader, thr=0.5):
    rles = []
    filename_and_class = []
    dataset = data_loader.dataset
    
    print("🚀 [Soft Voting] Ensemble Start...")
    
    with torch.no_grad():
        for images, image_names in tqdm(data_loader):
            images = images.to(DEVICE)
            image_name = image_names[0]
            
            final_prob = torch.zeros((29, 2048, 2048), device=DEVICE)
            total_weight = 0.0
            
            # --- 각 모델별 추론 및 가중 합산 ---
            for config in models_list:
                model = config['model']
                weight = config['weight']
                method = config['method']
                
                if weight <= 0: continue
                
                if method == 'sliding':
                    prob = sliding_window_inference(model, images)
                else: # method == 'full'
                    preds = model(images)
                    if isinstance(preds, tuple): preds = preds[0]
                    # 크기 보정
                    if preds.shape[-1] != 2048:
                        preds = F.interpolate(preds, size=(2048, 2048), mode="bilinear", align_corners=False)
                    prob = torch.sigmoid(preds).squeeze(0)
                
                final_prob += prob * weight
                total_weight += weight
            
            # 정규화 (Weights 합이 1이 아닐 경우 대비)
            if total_weight > 0:
                final_prob /= total_weight
                
            # --- [Re-flip] Right 손인 경우 복구 ---
            hand_side = dataset.hand_side_map.get(image_name, 'Unknown')
            if hand_side == 'Right':
                final_prob = torch.flip(final_prob, dims=[-1])
            
            # --- Thresholding ---
            preds = (final_prob > thr).detach().cpu().numpy()
            
            for c, segm in enumerate(preds):
                rle = encode_mask_to_rle(segm)
                rles.append(rle)
                filename_and_class.append(f"{IND2CLASS[c]}_{image_name}")
                
    return rles, filename_and_class

In [ ]:
if __name__ == '__main__':
    # 1. 모델 초기화 및 로드
    for config in MODEL_CONFIGS:
        print(f"Loading {config['name']}...")
        if config['type'] == 'sam2unet':
            model = SAM2UNet(HIERA_PATH).to(DEVICE)
        elif config['type'] == 'smp_unetpp':
            model = smp.UnetPlusPlus(encoder_name="efficientnet-b3", classes=29).to(DEVICE)
        
        # 체크포인트 로드 (state_dict 처리)
        if os.path.exists(config['path']):
            ckpt = torch.load(config['path'], map_location=DEVICE)
            if isinstance(ckpt, dict) and ('model' in ckpt or 'state_dict' in ckpt):
                state_dict = ckpt.get('model', ckpt.get('state_dict'))
            elif hasattr(ckpt, 'state_dict'): # 모델 객체 통째로 저장된 경우
                 state_dict = ckpt.state_dict()
            else:
                state_dict = ckpt
            
            model.load_state_dict(state_dict, strict=False)
            config['model'] = model # 로드된 모델 객체 저장
        else:
            print(f"❌ Path Not Found: {config['path']}")
            config['weight'] = 0 # 로드 실패시 가중치 0 처리

    # 2. 데이터셋
    tf = A.Compose([ToTensorV2()])
    test_ds = XRayInferenceDataset(IMAGE_ROOT, transforms=tf)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=4)

    # 3. 실행
    rles, fnames = ensemble_test(MODEL_CONFIGS, test_loader)
    
    classes, filename = zip(*[x.split("_") for x in fnames])
    df = pd.DataFrame({"image_name": [os.path.basename(f) for f in filename], "class": classes, "rle": rles})
    df.to_csv(SAVE_PATH, index=False)
    print(f"✅ Ensemble Result Saved to {SAVE_PATH}")